In this file the data will be cleaned and transfored for further analysis.
This will be done by connecting with the database and pass SQL querries through the connection.

At first, the required modules are imported:

In [1]:
import pandas as pd
import json
import mysql.connector
import os

Next, the credentials are loaded and the connection to the database is etablished:

In [2]:
#get credentials for MySQL/dbms
with open(r'C:\Users\mathi\Desktop\Datenanalyse\Credentials\mysql.json') as json_file:
    db_credentials = json.load(json_file)['MySQL']

#credentials as variables
host='localhost'
user=db_credentials['user']
password=db_credentials['password']
database='feederwatch'

#etablish connection to the dbms
mydb = mysql.connector.connect(
  host=host,
  user=user,
  password=password)

mycursor = mydb.cursor()

#select database
query=f'USE {database}'
mycursor.execute(query)

Now we'll check if all the tables are availible and how they are named.

In [7]:
query='SHOW TABLES'
mycursor.execute(query)
for table in mycursor:
    print(table)

('pfw_1988_1995_public',)
('pfw_1996_2000_public',)
('pfw_2001_2005_public',)
('pfw_2006_2010_public',)
('pfw_2011_2015_public',)
('pfw_2016_2020_public',)
('pfw_2021_public',)
('pfw_count_site_data_public_2021',)


Because the names are just the ones from the original files, we can rename them:

In [8]:
query='''RENAME TABLE
        pfw_1988_1995_public TO pfw_1988,
        pfw_1996_2000_public TO pfw_1996,
        pfw_2001_2005_public TO pfw_2001,
        pfw_2006_2010_public TO pfw_2006,
        pfw_2011_2015_public TO pfw_2011,
        pfw_2016_2020_public TO pfw_2016,
        pfw_2021_public TO pfw_2021,
        pfw_count_site_data_public_2021 TO pfw_site_data'''

mycursor.execute(query)

query='SHOW TABLES'
mycursor.execute(query)
for table in mycursor:
    print(table)

('pfw_1988',)
('pfw_1996',)
('pfw_2001',)
('pfw_2006',)
('pfw_2011',)
('pfw_2016',)
('pfw_2021',)
('pfw_site_data',)


Because we expect that the tables with datafrom different years have the same structer, they are grouped together to acess them easily with loops later on. 

In [3]:
observation_tables=set()
query='SHOW TABLES'
mycursor.execute(query)
for table in mycursor:
    observation_tables.add(table[0])

observation_tables.discard('pfw_site_data') #removes site_data from set because it should be treated differently

print(observation_tables)

{'pfw_1996', 'pfw_2001', 'pfw_1988', 'pfw_2021', 'pfw_2016', 'pfw_2011', 'pfw_2006'}


Because the observation tables should look all the same and contain a timestamp, we should be able to merge all these tables into a bigger one. Before this step is performed, we should control, if they all have the same column names and order.

In [7]:
column_names=pd.DataFrame()

for table in observation_tables:
    query=f'DESCRIBE {table}'
    mycursor.execute(query)
    columns=[]
    for line in mycursor:
        columns.append(line[0])
    
    temp_df=pd.DataFrame({table: columns})
    column_names=pd.concat([column_names, temp_df], axis=1)

print(column_names)

              pfw_1996            pfw_2001            pfw_1988  \
0                index               index               index   
1               LOC_ID              LOC_ID              LOC_ID   
2             LATITUDE            LATITUDE            LATITUDE   
3            LONGITUDE           LONGITUDE           LONGITUDE   
4    SUBNATIONAL1_CODE   SUBNATIONAL1_CODE   SUBNATIONAL1_CODE   
5      ENTRY_TECHNIQUE     ENTRY_TECHNIQUE     ENTRY_TECHNIQUE   
6               SUB_ID              SUB_ID              SUB_ID   
7               OBS_ID              OBS_ID              OBS_ID   
8                Month               Month               Month   
9                  Day                 Day                 Day   
10                Year                Year                Year   
11      PROJ_PERIOD_ID      PROJ_PERIOD_ID      PROJ_PERIOD_ID   
12        SPECIES_CODE        SPECIES_CODE        SPECIES_CODE   
13            HOW_MANY            HOW_MANY            HOW_MANY   
14        

Because the output exceeds the size limit of a jupyter notbook cell, visual inspection won't work. 
Instead, a boolen matrix can be generated, where all column names are compared with each other.

In [10]:
as_np_array = column_names.values
comparison_matrix = pd.DataFrame((as_np_array.T[:, None] == as_np_array.T).all(-1), column_names.columns, column_names.columns)
print(comparison_matrix)

          pfw_1996  pfw_2001  pfw_1988  pfw_2021  pfw_2016  pfw_2011  pfw_2006
pfw_1996      True      True      True     False      True      True      True
pfw_2001      True      True      True     False      True      True      True
pfw_1988      True      True      True     False      True      True      True
pfw_2021     False     False     False     False     False     False     False
pfw_2016      True      True      True     False      True      True      True
pfw_2011      True      True      True     False      True      True      True
pfw_2006      True      True      True     False      True      True      True


From the comparison we can tell, that all tables have the same column names except 'pfw_2021'. For further investigation, we'll display the column names of 'pfw_2021' next to the names from another table.

In [13]:
column_names[['pfw_2021', 'pfw_1988']]

,pfw_2021,pfw_1988
0,index,index
1,loc_id,LOC_ID
2,latitude,LATITUDE
3,longitude,LONGITUDE
4,subnational1_code,SUBNATIONAL1_CODE
5,entry_technique,ENTRY_TECHNIQUE
6,sub_id,SUB_ID
7,obs_id,OBS_ID
8,Month,Month
9,Day,Day


A obvious differnce is, that they are written in differnt casses (lower/upper). But because the case dosn't matter for merging the tables, we can ignore this difference. 
Another difference is visible in row 14, where one column s labeled 'valid' and the other 'PLUS_CODE'. It seems, that the column 'plus_code' is missing in 'pfw_2021', which leads to a shift and missmatch in all following columns and a NaN column at the end to fill up the data.
To figure out, what the missing column 'plus_code' should ontain, we can check the FeederWatch_Data_Dictonary.xlsx provided by Project FeederWatch. 
According to this documentation it should contain a binary value:
>"Variable indicating if the total number of a species seen was larger than the value reported".
Even though this variable dosn't seem to be very interesting for analysis, we can keep it and create an empty column for the table, where it's missing.  

In [15]:
query='''ALTER TABLE pfw_2021
        ADD COLUMN plus_code BINARY NULL
        AFTER how_many'''
mycursor.execute(query)

To confirm the change, we can run code from above again. 

In [16]:
column_names=pd.DataFrame()

for table in observation_tables:
    query=f'DESCRIBE {table}'
    mycursor.execute(query)
    columns=[]
    for line in mycursor:
        columns.append(line[0])
    
    temp_df=pd.DataFrame({table: columns})
    column_names=pd.concat([column_names, temp_df], axis=1)

column_names[['pfw_2021', 'pfw_1988']]

,pfw_2021,pfw_1988
0,index,index
1,loc_id,LOC_ID
2,latitude,LATITUDE
3,longitude,LONGITUDE
4,subnational1_code,SUBNATIONAL1_CODE
5,entry_technique,ENTRY_TECHNIQUE
6,sub_id,SUB_ID
7,obs_id,OBS_ID
8,Month,Month
9,Day,Day
